Import

In [1]:
!git clone https://github.com/AndreasHammerKU/CardiacCTAnalysis.git
%cd CardiacCTAnalysis
!git checkout add_frame_history

Cloning into 'CardiacCTAnalysis'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 277 (delta 17), reused 22 (delta 10), pack-reused 223 (from 1)
Receiving objects: 100% (277/277), 1.66 MiB | 25.41 MiB/s, done.
Resolving deltas: 100% (143/143), done.
/content/CardiacCTAnalysis
Branch 'add_frame_history' set up to track remote branch 'add_frame_history' from 'origin'.
Switched to a new branch 'add_frame_history'


Imports from Github Repository

In [2]:
!pip install dash
import numpy as np

# Custom Imports
import utils.io_utils as io
import utils.logger as logs
from baseline.BaseEnvironment import MedicalImageEnvironment
from utils.io_utils import DataLoader
from baseline.BaseAgent import DQNAgent

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

dataset_folder = '/content/drive/MyDrive/Data'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0
Mounted at /content/drive


Hyperparameters

In [3]:
MAX_STEPS = 500
EPISODES = 40
IMAGE_INTERVAL = 2
AGENTS = 6
EVALUATION_STEPS = 30
DECAY = 200 # EPSILON = MIN_EPS - (MAX_EPS - MIN_EPS) * e^(-1 * current_step / decay)

In [4]:
debug = False

# Colab not enough RAM
preload_images = False
logger = logs.setup_logger(debug)

dataLoader = DataLoader(dataset_folder)

Training

In [6]:
# Initialize training environment
train_env = MedicalImageEnvironment(logger=logger,
                              dataLoader=dataLoader,
                              image_list=['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15', 'n16', 'n17', 'n18', 'n19', 'n20', 'n21', 'n22', 'n23', 'n24', 'n25', 'n26', 'n27', 'n28', 'n29', 'n30'],
                              agents=AGENTS,
                              preload_images=preload_images)
eval_env = MedicalImageEnvironment(logger=logger,
                              task="eval",
                              dataLoader=dataLoader,
                              image_list=['n31', 'n32', 'n33', 'n34', 'n35', 'n36', 'n37', 'n38', 'n39', 'n40'],
                              agents=AGENTS)
agent = DQNAgent(train_environment=train_env,
                 eval_environment=eval_env,
                 task="train",
                 logger=logger,
                 state_dim=train_env.state_size,
                 action_dim=train_env.n_actions,
                 agents=AGENTS,
                 max_steps=MAX_STEPS,
                 episodes=EPISODES,
                 decay=DECAY,
                 image_interval=IMAGE_INTERVAL,
                 evaluation_steps=EVALUATION_STEPS)

agent.train_dqn()

INFO:Logger:Episode 1: Total Reward = -17.05 | Final Avg Distance 45.80 | All Reached Goal False | Avg Closest Point = 8.09 | Avg Furthest Point = 57.87
INFO:Logger:Episode 2: Total Reward = -3.57 | Final Avg Distance 32.32 | All Reached Goal False | Avg Closest Point = 8.03 | Avg Furthest Point = 43.44
INFO:Logger:Episode 3: Total Reward = 6.92 | Final Avg Distance 26.76 | All Reached Goal False | Avg Closest Point = 8.75 | Avg Furthest Point = 46.53
INFO:Logger:Episode 4: Total Reward = 11.72 | Final Avg Distance 21.96 | All Reached Goal False | Avg Closest Point = 5.21 | Avg Furthest Point = 44.09
INFO:Logger:Episode 5: Total Reward = -35.59 | Final Avg Distance 60.90 | All Reached Goal False | Avg Closest Point = 8.01 | Avg Furthest Point = 64.10
INFO:Logger:Episode 6: Total Reward = -28.80 | Final Avg Distance 54.11 | All Reached Goal False | Avg Closest Point = 7.84 | Avg Furthest Point = 60.07
INFO:Logger:Episode 7: Total Reward = 1.31 | Final Avg Distance 37.51 | All Reached Go

KeyboardInterrupt: 

In [ ]:
train_env.visualize_current_state()

In [ ]:
eval_env.visualize_current_state()

Evaluate

In [ ]:
test_env = MedicalImageEnvironment(logger=logger,
                              task="test",
                              dataLoader=dataLoader,
                              image_list=['n41', 'n42', 'n43', 'n44', 'n45', 'n46', 'n47', 'n48', 'n49', 'n50'],
                              agents=AGENTS)
agent = DQNAgent(train_environment=train_env,
                 eval_environment=eval_env,
                 test_environment=test_env,
                 task="test",
                 logger=logger,
                 state_dim=test_env.state_size,
                 action_dim=test_env.n_actions,
                 agents=AGENTS,
                 model_path="latest-model.pt",
                 max_steps=MAX_STEPS,
                 episodes=EPISODES,
                 evaluation_steps=25
                 )

agent.test_dqn()

In [ ]:
test_env.visualize_current_state()